In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
from selenium.webdriver.chrome.options import Options
import time
import dateutil.parser as parser
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")



options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def scrollDown(browser, numberOfScrollDowns):
    body = browser.find_element_by_tag_name("body")
    while numberOfScrollDowns >=0:
        body.send_keys(Keys.PAGE_DOWN)
        numberOfScrollDowns -= 1
        # print(numberOfScrollDowns)
    return browser

def clean_date(date):
  if date=='':
    return '',''
  else:
    if '-' not in date:
      try:
        d = parser.parse(date)
        d = datetime.strftime(d, "%m/%d/%Y")
        return d,d
      except:
        return date,date
    elif '-' in date:
      s = date.split()
      if len(s)==5:
        try:
          d1 = s[0]+' '+s[1]+' '+s[4]
          d2 = s[0]+' '+s[3][:-1]+' '+s[4]
          d1 = datetime.strftime(parser.parse(d1), "%m/%d/%Y")
          d2 = datetime.strftime(parser.parse(d2), "%m/%d/%Y")
          return d1,d2
        except:
          return date,date
      elif len(s)==6:
        try:
          d1 = s[0]+' '+s[1]+' '+s[5]
          d2 = s[3]+' '+s[4][:-1]+' '+s[5]
          d1 = datetime.strftime(parser.parse(d1), "%m/%d/%Y")
          d2 = datetime.strftime(parser.parse(d2), "%m/%d/%Y")
          return d1,d2
        except:
          return date,date
      elif len(s)==7:
        try:
          d1 = s[0]+' '+s[1][:-1]+' '+s[2]
          d2 = s[4]+' '+s[5][:-1]+' '+s[6]
          d1 = datetime.strftime(parser.parse(d1), "%m/%d/%Y")
          d2 = datetime.strftime(parser.parse(d2), "%m/%d/%Y")
          return d1,d2
        except:
          return date,date
    return date,date



with open('./hackathons_data.csv', 'w') as f:
    # create the csv writer
    writer = csv.writer(f)
    header = ['Location','Title', 'start_date','end_date', 'Prize Money','Participants','Host','Themes']
    writer.writerow(header)
    
    url = 'https://devpost.com/hackathons'
    driver = webdriver.Chrome('chromedriver',options=options)
    driver.get(url)
    
    for i in range(700):
      driver = scrollDown(driver, 9)
    hacks = driver.find_elements_by_class_name('hackathon-tile')
    
    for hack in hacks:
        c = hack.find_element_by_xpath("./child::*").text
        if c!='FEATURED':
          host = ""
          try:
            host = hack.find_element_by_class_name('host-label').text
          except:
            pass
          themes=""
          try:
            themeBlock = hack.find_element_by_class_name('themes')
            try:
              themesElements = themeBlock.find_elements_by_class_name('theme-label')
              for theme in themesElements:
                themes+=theme.text
                themes+='#'
              themes=themes[:-1]
            except:
              themes = ''
          except:
            pass
          c = c.split('\n')[:6]
          row = c[:2]
          # time_status = clean_time(c[2])  # not needed

          start_date, end_date = clean_date(c[3])
          p_idx = c[4].find('in')
          prize = c[4][:p_idx-1]
          participants = c[5].split(' ')[0]
          # row.append(time_status)  # not needed
          row.append(start_date)
          row.append(end_date)
          row.append(prize)
          row.append(participants)
          row.append(host)
          row.append(themes)

          writer.writerow(row)

